In [1]:
from utils import non_weather_training_columns_list, with_weather_training_columns_list
import pandas as pd
import pickle 

pd.set_option('display.max_columns', None)

In [6]:
def forecast_next_24_hours_output_flow_rate(year, month, day, hour):
    
    input_df = pd.read_parquet("../data/silver/training_dataset.parquet")
    timestamp = pd.Timestamp(year=year, month=month, day=day, hour=hour)
    input_df = input_df[input_df["timestamp"] == timestamp]
    
    X = input_df[non_weather_training_columns_list]
    X_weather = input_df[with_weather_training_columns_list]
    
    predictions = []
    for i in range(1, 25):
        new_prediction = {}
        next_timestamp = timestamp + pd.Timedelta(hours=i)
        model = pickle.load(open(f"../models/xgb_{i}h.pkl", "rb")) # aqui tem que mudar, esse path vai dar erro
        new_prediction['timestamp'] = next_timestamp
        new_prediction['forecasted_output_flow_rate'] = round(float(model.predict(X)[0]), 2)
        predictions.append(new_prediction)


    weather_predictions = []
    for i in range(1, 25):
        new_prediction = {}
        next_timestamp = timestamp + pd.Timedelta(hours=i)
        model = pickle.load(open(f"../models/xgb_with_weather_{i}h.pkl", "rb")) # aqui tem que mudar, esse path vai dar erro
        new_prediction['timestamp'] = next_timestamp
        new_prediction['weather_forecasted_output_flow_rate'] = round(float(model.predict(X_weather)[0]), 2)
        weather_predictions.append(new_prediction)
        
    predictions = pd.DataFrame(predictions)
    weather_predictions = pd.DataFrame(weather_predictions)
    merged_df = pd.merge(predictions, weather_predictions, on='timestamp')
    merged_df.to_parquet("../data/gold/question_4_and_7_gold.parquet")

In [9]:
forecast_next_24_hours_output_flow_rate(2023, 8, 18, 18)

In [10]:
q4 = pd.read_parquet("../data//gold/question_4_and_7_gold.parquet")
q4

,timestamp,forecasted_output_flow_rate,weather_forecasted_output_flow_rate
0,2023-08-18 19:00:00,36.69,36.67
1,2023-08-18 20:00:00,34.20,35.40
2,2023-08-18 21:00:00,35.11,36.26
3,2023-08-18 22:00:00,39.27,39.27
4,2023-08-18 23:00:00,39.70,39.70
5,2023-08-19 00:00:00,39.54,39.54
6,2023-08-19 01:00:00,39.52,39.52
7,2023-08-19 02:00:00,39.44,39.44
8,2023-08-19 03:00:00,39.54,39.54
9,2023-08-19 04:00:00,39.46,39.46
